In [2]:
!pip install tensorflow
!pip install deepface
!pip install opencv-python
!pip install spotipy

In [2]:
#import the modules
import os
from dotenv import load_dotenv
import pandas as pd
import tensorflow as tf
import cv2
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from deepface import DeepFace

In [ ]:
scope = "user-read-playback-state user-top-read user-modify-playback-state user-follow-read"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id = os.getenv('SPOTIPY_CLIENT_ID'),
                                               client_secret = os.getenv('SPOTIPY_CLIENT_SECRET'),
                                               redirect_uri = os.getenv('SPOTIPY_REDIRECT_URI'),
                                               scope=scope))


In [ ]:
auth_url = sp.auth_manager.get_authorize_url()
print(f"Go to the following URL to authenticate: {auth_url}")

In [ ]:
user = sp.current_user()
print(user)

In [ ]:
emotion_genre = {
    'happy': {'min_tempo': 120, 'max_tempo': 150, 'min_energy': 0.7, 'max_energy': 1.0},
    'sad': {'min_tempo': 60, 'max_tempo': 90, 'min_energy': 0.2, 'max_energy': 0.5},
    'calm': {'min_tempo': 60, 'max_tempo': 80, 'min_energy': 0.2, 'max_energy': 0.4},
    'angry': {'min_tempo': 90, 'max_tempo': 130, 'min_energy': 0.7, 'max_energy': 1.0},
    'surprise': {'min_tempo': 120, 'max_tempo': 150, 'min_energy': 0.7, 'max_energy': 1.0},
    'neutral': {'min_tempo': 60, 'max_tempo': 100, 'min_energy': 0.4, 'max_energy': 0.8},
    'fear': {'min_tempo': 60, 'max_tempo': 90, 'min_energy': 0.2, 'max_energy': 0.5}
}

In [ ]:
# function to get songs based on emotion
def get_songs(emotion):
    # Get features based on the emotion
    features = emotion_genre.get(emotion, {})

    # Fetch song recommendations based on genre and features (tempo, energy, etc.)
    results = sp.recommendations(
        seed_genres=['pop', 'rnb', 'rock', 'acoustic', 'metal'],
        limit=20,
        min_tempo=features.get('min_tempo', 60),  # Set default values if features are empty
        max_tempo=features.get('max_tempo', 150),
        min_energy=features.get('min_energy', 0.1),
        max_energy=features.get('max_energy', 1.0)
    )

    # Print the suggested songs
    print(f"Songs for {emotion} mood:")
    for i, track in enumerate(results['tracks']):
        print(f"{i+1}: {track['name']} by {track['artists'][0]['name']}")


In [ ]:
#function to detect emotion
def get_emotion():
  #from google.colab.patches import cv2_imshow
  cap = cv2.VideoCapture(0)
  ret, frame = cap.read()

  if not ret or frame is None:
    print("cant find face")

  else:
    try:
      result  = DeepFace.analyze(frame, actions = ['emotion'])
      emotion = result[0]['dominant_emotion']
      print(emotion)
    except:
      print("cant detect emotion")

  cv2.waitKey(0)
  cv2.destroyAllWindows()
  return emotion

In [ ]:
emotion = get_emotion()
get_songs(emotion)